In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import glob
from google.colab.patches import cv2_imshow

In [273]:
round_num = 6

In [544]:
def run(frames, x, y, w, h, tracking_alg, path):
    hei, wei = frames[0].shape
    start_point = (x, y)
    color = (255, 0, 0)
    end_point = (x + w, y + h)
    # print(start_point, end_point)
    f = np.stack((frames[0],)*3, axis=-1)
    f = np.ascontiguousarray(f, dtype=np.uint8)
    image = cv2.rectangle(f, start_point, end_point, color, 1)
    cv2.imwrite( path + str(0) + '.png', f)
    for i in range(len(frames) - 1):
        f1 = frames[i]
        f2 = frames[i+1]
        f1 = np.ascontiguousarray(f1, dtype=np.uint8)
        f2 = np.ascontiguousarray(f2, dtype=np.uint8)
        # print(i, x, y, w, h)
        T = f1[y:y+h, x:x+w]
        nx, ny, nw, nh = tracking_alg(T, f2,  [x, y, w, h])
        if nx > 0 and ny > 0 and nw > 0 and nh > 0:
            x, y, w, h = nx, ny, nw, nh
        if h+y > hei:
            y -= abs(h+y - hei)
        if w+x > wei:
            x -= abs(w+x - wei)
        if y < 0:
            y += abs(y)
        if x < 0:
            x += abs(x)
        start_point = (x, y)
        color = (255, 0, 0)
        end_point = (x + w, y + h)
        f = np.stack((f2,)*3, axis=-1)
        image = cv2.rectangle(f, start_point, end_point, color, 1)
        cv2.imwrite( path + str(i+1) + '.png', f)
        


In [545]:
def create_video(folder_path, output_name):
    img_array = []
    for filename in glob.glob(folder_path+'*.png'):
        # print(filename)
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width, height)
        img_array.append(img)
    print(len(img_array))
    out = cv2.VideoWriter(output_name+'.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 1, size)
    
    for i in range(len(img_array)):
        out.write(img_array[i])
    out.release()

In [620]:
def delta_p_trans(B, wgx, wgy, corners):
  H = 0
  sec_p = 0
  for i in corners:
    x ,y = i.ravel()
    w = B[y][x]
    dx_dy = np.expand_dims(np.array([wgx[y][x], wgy[y][x]]), axis=0)
    sec_p += dx_dy.T * w
    H += np.matmul(dx_dy.T, dx_dy)
  return np.matmul(np.linalg.inv(H), sec_p)

In [604]:

def kl_trans(T, nframe, box, eps=1e-4, max_iter=150):
  tr, tc = T.shape
  sx, sy, ex, ey = box[0], box[1], box[0] + box[2], box[1] + box[3]
  p = np.zeros(2)
  I = nframe[sy:ey, sx:ex]
  corner_count = 1
  for i in range(max_iter):
    warp = np.array([[1, 0, p[0]], [0, 1 , p[1]]])
    rows, cols = nframe.shape
    WnF = cv2.warpAffine(nframe, warp, (cols, rows))
    W = WnF[sy:ey, sx:ex]
    B = T.astype(int) - W.astype(int)
    gx = cv2.Sobel(nframe, cv2.CV_64F, 1, 0, ksize=5)
    gy = cv2.Sobel(nframe, cv2.CV_64F, 0, 1, ksize=5)
    cgx = cv2.warpAffine(gx, warp, (cols, rows)) 
    cgy = cv2.warpAffine(gy, warp, (cols, rows)) 
    wgx = cgx[sy:ey, sx:ex]
    wgy = cgy[sy:ey, sx:ex]
    corners = cv2.goodFeaturesToTrack(W, 50, 0.01, 10, useHarrisDetector = True)
    corners = np.int0(corners)
    dp = delta_p_trans(B, wgx.astype(int), wgy.astype(int), corners)
    normv = round(np.linalg.norm(dp), round_num)
    p += dp.reshape(p.shape)
    if normv <= eps:
      # print(normv, i)
      return p
  # print(normv, i)
  return p

In [605]:
def plot_trans(T, nframe, box):
  tr, tc = T.shape
  print('Template ')
  cv2_imshow(T)

  sx, sy, ex, ey = box[0], box[1], box[0] + box[2], box[1] + box[3]
  p = np.zeros(6)
  I = nframe[sy:ey, sx:ex]
  warp = np.array([[1 + p[0], p[1], p[2]], [p[3], 1 + p[4], p[5]]])
  rows, cols = nframe.shape
  WnF = cv2.warpAffine(nframe, warp, (cols, rows))
  W = WnF[sy:ey, sx:ex]
  print('Warped Iamge')
  cv2_imshow(W)
  B = T - W
  print('Error Diff. between Templat and Warped Image')
  cv2_imshow(B)
  gx = cv2.Sobel(nframe, cv2.CV_64F, 1, 0, ksize=5)
  gy = cv2.Sobel(nframe, cv2.CV_64F, 0, 1, ksize=5)
  cgx = cv2.warpAffine(gx, warp, (cols, rows)) 
  cgy = cv2.warpAffine(gy, warp, (cols, rows)) 
  wgx = cgx[sy:ey, sx:ex]
  wgy = cgy[sy:ey, sx:ex]
  print('Derv. of X')
  cv2_imshow(wgx)
  print('Derv. of Y')
  cv2_imshow(wgy)
  IW0 = wgx 
  IW1 = wgy
  print('Derv. of IW0')
  cv2_imshow(IW0)
  print('Derv. of IW1')
  cv2_imshow(IW1)



# Car 1

In [ ]:
frames = np.load('car1.npy')
frames.shape

In [ ]:
frames = np.moveaxis(frames, -1, 0)
frames.shape

In [ ]:
frames = np.load('car1.npy')
frames = np.moveaxis(frames, -1, 0)
path = r'C:\Users\zayton\Downloads\car1\\'


In [ ]:
# def tracking_affine_car1(cframe, nframe, box):
#   sx, sy, ex, ey = box[0], box[1], box[0] + box[3], box[1] + box[2]
#   p = kl_affine(cframe, nframe, box, eps=1e-2, max_iter=100)
#   warp_mat = np.array([[1 + p[0], p[1], p[2]], [p[3], 1 + p[4], p[5]]])
#   rect_pts = np.array([[sx, ex], [sy, ey], [1, 1]])
#   new_rect_pts = np.matmul(warp_mat, rect_pts)
#   return int(round(new_rect_pts[0][0], 0)), int(round(new_rect_pts[1][0], 0)), int(round(new_rect_pts[1][1] - new_rect_pts[1][0], 0)), int(round(new_rect_pts[0][1] - new_rect_pts[0][0], 0))

In [ ]:
def tracking_trans_car1(cframe, nframe, box):
  sx, sy, ex, ey = box[0], box[1], box[0] + box[2], box[1] + box[3]
  p = kl_trans(cframe, nframe, box, eps=1e-2, max_iter=100)
  warp_mat = np.array([[1, 0, p[0] + 0.5], [0, 1 , p[1]]])
  # print(p[0], warp_mat)
  rect_pts = np.array([[sx, ex], [sy, ey], [1, 1]])
  new_rect_pts = np.matmul(warp_mat, rect_pts)
  return int(round(new_rect_pts[0][0], 0)), int(round(new_rect_pts[1][0], 0)), int(round(new_rect_pts[0][1] - new_rect_pts[0][0], 0)), int(round(new_rect_pts[1][1] - new_rect_pts[1][0], 0))

In [ ]:
len(frames)

In [ ]:
x, y, w, h = 100, 100, 250, 180

In [ ]:
run(frames=frames, x=x, y=y, w=w, h=h, tracking_alg=tracking_trans_car1, path=path)


In [ ]:
create_video(r'car1/', 'car1_trans1')


# Car 2 

In [169]:
frames = np.load('car2.npy')
frames = np.moveaxis(frames, -1, 0)
round_num = 6

In [170]:
len(frames)

415

In [171]:
path = r'C:\Users\zayton\Downloads\car2\\'

In [281]:
def tracking_trans_car2(cframe, nframe, box):
  sx, sy, ex, ey = box[0], box[1], box[0] + box[2], box[1] + box[3]
  p = kl_trans(cframe, nframe, box, eps=1e-2, max_iter=100)
  warp_mat = np.array([[1, 0, p[0]+0.1], [0, 1 , p[1] - 0.4]])
  # print(warp_mat)
  rect_pts = np.array([[sx, ex], [sy, ey], [1, 1]])
  new_rect_pts = np.matmul(warp_mat, rect_pts)
  return int(round(new_rect_pts[0][0], 0)), int(round(new_rect_pts[1][0], 0)), int(round(new_rect_pts[0][1] - new_rect_pts[0][0], 0)), int(round(new_rect_pts[1][1] - new_rect_pts[1][0], 0))

In [282]:
x, y, w, h = 25, 80, 150, 80

In [283]:
run(frames=frames, x=x, y=y, w=w, h=h, tracking_alg=tracking_trans_car2, path=path)


In [321]:
create_video(r'car2/', 'car2_tran1')


# Landing

In [ ]:
frames = np.load('landing.npy')
frames = np.moveaxis(frames, -1, 0)

In [ ]:
path = r'C:\Users\zayton\Downloads\landing\\'


In [ ]:
def tracking_trans_landing(cframe, nframe, box):
  sx, sy, ex, ey = box[0], box[1], box[0] + box[2], box[1] + box[3]
  p = kl_trans(cframe, nframe, box, eps=1e-2, max_iter=100)
  warp_mat = np.array([[1, 0, p[0]-0.8], [0, 1 , p[1]+1.5]])
  # print(p[0], warp_mat)
  rect_pts = np.array([[sx, ex], [sy, ey], [1, 1]])
  new_rect_pts = np.matmul(warp_mat, rect_pts)
  return int(round(new_rect_pts[0][0], 0)), int(round(new_rect_pts[1][0], 0)), int(round(new_rect_pts[0][1] - new_rect_pts[0][0], 0)), int(round(new_rect_pts[1][1] - new_rect_pts[1][0], 0))

In [ ]:
len(frames)


In [ ]:
x, y, w, h = 400, 40, 250, 200

In [ ]:
run(frames=frames, x=x, y=y, w=w, h=h, tracking_alg=tracking_trans_landing, path=path)

In [ ]:
create_video(r'landing/', 'landing_tran1')